In [1]:
import numpy as np
from scipy.optimize import curve_fit
from scipy.integrate import trapezoid
from random import gauss

In [2]:
def EW_AdjGauss(wl,fl,cf,mins,maxs, n_iter):

    wdmmn = np.where(np.abs(wl[mins]-cf) == min(np.abs(wl[mins]-cf)))[0][0]
    wdmmx = np.where(np.abs(wl[maxs]-cf) == min(np.abs(wl[maxs]-cf)))[0][0]

    tp = [mins[wdmmn], min(np.abs(wl[mins]-cf)), maxs[wdmmx], min(np.abs(wl[maxs]-cf))]

    if tp[1]<tp[3]:
        
        p = mins[wdmmn]
        idx = []
        i,j = 0,0
        
        while fl[p + i] < fl[p + i + 1]:           
            i+=1
            idx.append(p + i)

        idx.append(p)

        while fl[p - j] < fl[p - j - 1]:            
            j+=1
            idx.append(p - j)
            
    else: 
        
        p= maxs[wdmmx]
        idx = []
        i,j = 0,0
        
        while fl[p + i] > fl[p + i + 1]:            
            i+=1
            idx.append(p + i)

        idx.append(p)

        while fl[p - j] > fl[p - j - 1]:            
            j+=1
            idx.append(p - j)

    idx.sort()
    idx

    if len(idx)<= 3:

        return [999991,999991,999991,999991,999991,999991,999991]


    else:
        try:
            def function (x, x_0, a, b):
                    return   a * (np.exp(-b*(x-x_0)**2))
            
            fl_p = fl[p]
            wl_p = wl[p]        
            x = wl[idx]
            y = fl[idx]
            
            EWg_ = []
            cfS_ = []
            sS_ = []
            for i in np.arange(0,n_iter):
                yg = [gauss(j, np.sqrt(j)) for j in y]
                
                if np.isnan(yg).any():
                    continue
                else:
                    ycg = (yg[-1] - yg[0])/(x[-1] - x[0]) * (x - x[0]) + yg[0]
                    ycg_f = ycg[np.where(idx == p)][0]
                    yg_sc = yg-ycg
                    res, cov = curve_fit(function, x, yg_sc, p0=[wl_p,fl_p,1])
                    flux = trapezoid(yg_sc,x)
                    EW = -flux/ycg_f
                    EWg_.append(EW)
                    cfS_.append(res[0])
                    sS_.append(np.sqrt(1/(2 * res[2])))
        except RuntimeError as error:
            if "Number of calls to function has reached maxfev" in str(error):
                pass
        
        sS_ar = np.array(sS_)[~np.isnan(sS_)]
        EW = np.mean(np.array(EWg_)[~np.isnan(sS_)])
        eEW = np.std(np.array(EWg_)[~np.isnan(sS_)])
        cf_a = np.mean(np.array(cfS_)[~np.isnan(sS_)])
        ecf_a = np.std(np.array(cfS_)[~np.isnan(sS_)])
        s = np.mean(np.array(sS_)[~np.isnan(sS_)])
        es = np.std(np.array(sS_)[~np.isnan(sS_)])  # gfwhm = 2√2ln2 s
        return [cf_a,ecf_a,EW,eEW,2.355*s, 2.355*es, len(np.array(sS_)[~np.isnan(sS_)])]